<div style="display: grid; grid-template-columns: repeat(2, 1fr); align-items: end;">
    <div>
        
    </div>
    <div>
        <p style="margin: 0; text-align:right;">22.401 · Fonaments de Programació</p>
        <p style="margin: 0; text-align:right;">Grau en Ciència de Dades Aplicada</p>
        <p style="margin: 0; text-align:right;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
    </div>
</div>

Fonaments de Programació
============================

---
Unitat 5: Adquisició de dades en Python
-----------------------------------------------------
---

# Instruccions d'ús

Aquest document és un *notebook* interactiu que intercala explicacions més aviat teòriques de conceptes de programació amb fragments de codi executables. Per aprofitar els avantatges que aporta aquest format, us recomanem que, en primer lloc, llegiu les explicacions i el codi que us proporcionem. D'aquesta manera tindreu un primer contacte amb els conceptes que hi exposem. Ara bé, **la lectura és només el principi!** Una vegada hàgiu llegit el contingut, no oblideu executar el codi proporcionat i modificar-lo per crear-ne variants que us permetin comprovar que heu entès la seva funcionalitat i explorar-ne els detalls d'implementació. Per últim, us recomanem també consultar la documentació enllaçada per explorar amb més profunditat les funcionalitats dels mòduls presentats.

Per tal de desar possibles modificacions que feu sobre aquest notebook, us aconsellem que munteu la unitat de Drive a Google Colaboratory (colab). Per fer-ho, heu d'executar les següents instruccions:

In [ ]:
import os

VERIFICA_ESTIL = False

if os.getenv("COLAB_RELEASE_TAG"):
    # munta la unitat de google drive i accedeix a la carpeta de la unitat
    from google.colab import drive
    drive.mount('/content/drive')
    %cd "/content/drive/MyDrive/Colab Notebooks/python_datascience/python_1/unit_5"

# verificació de PEP8
if VERIFICA_ESTIL:
    if os.getenv("COLAB_RELEASE_TAG"):
        print("instal·lant flake8, pycodestyle i pycodestyle_magic...")
        %pip install pycodestyle flake8 pycodestyle_magic
    %load_ext pycodestyle_magic
    %pycodestyle_on

# 1 Introducció

Els processos d'adquisició de dades són molt diversos. En aquesta unitat, veurem exemples d'adquisició de dades d'internet amb tres mètodes diferents:

-   descàrrega directa
-   petició a APIs de tercers
-   *web crawling*

Pel que respecta a la interacció amb APIs de tercers, repassarem dues alternatives, la construcció manual de les peticions HTTP i l'ús de biblioteques Python.

En relació amb el _web crawling_, veurem com utilitzar la biblioteca [Scrapy](https://scrapy.org/) per construir un petit _web crawler_ que capturi dades del nostre interès.

# 2 Descàrrega directa de dades

La descàrrega directa del conjunt de dades és potser el mètode més senzill d'adquisició de dades i consisteix a descarregar un fitxer amb les dades d'interès ja recopilades per algun altre analista. De fet, a la unitat anterior ja hem fet servir aquest mètode per adquirir el fitxer amb les dades sobre els personatges de còmic de Marvel. Un cop
descarregat el fitxer, el procediment per carregar-lo en Python dependrà del format concret (ja hem vist un exemple de càrrega de dades des d'un fitxer .csv).

Alguns dels llocs web on podeu trobar conjunts de dades a analitzar són:
- [Open Data gencat](http://dadesobertes.gencat.cat/), el portal de dades obertes de la Generalitat.
- [datos.gov.es](http://datos.gob.es/es/catalogo), el catàleg de conjunts de dades del Govern d'Espanya.
- [European Data Sources](https://data.europa.eu/), el portal de dades obertes de la Unió Europea.
- [Mark Newman network datasets](http://www-personal.umich.edu/~mejn/netdata/), conjunts de dades en forma de xarxa recopilats per Mark Newman.
- [Stanford Large Network Dataset Collection](http://snap.stanford.edu/data/), un altre recopilatori de conjunts de dades en forma de xarxa, en aquest cas creat per Jure Leskovec.
- [SecRepo.com](http://www.secrepo.com/), dades relacionades amb la seguretat.
- [AWS Public Datasets](https://aws.amazon.com/public-datasets/), conjunts de dades recopilades i hostatjades per Amazon.
- [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/), dades recopilades per un grup de recerca de la Universitat de Califòrnia, Irvine.
- El [dipòsit de Five Thirty Eight](https://github.com/fivethirtyeight), que recull dades utilitzades a articles de la publicació i que ja hem vist a la unitat anterior.

# 3 Petició a les API de tercers

En aquest apartat definirem què és una API, descriurem les peticions i respostes HTTP que s'utilitzen per interactuar amb les API web i, finalment, presentarem els dos formats de representació de dades més utilitzades per les API web.

## 3.1 El concepte d'API

Una **API** (de l'anglès, *Application Programming Interface*) és un conjunt de mètodes de comunicació entre diversos components de programari.

Les API faciliten el treball d'integració de programes, ja que permeten oferir una interfície clara i ben especificada amb la qual interactuar amb una aplicació, amagant-ne els detalls de la implementació i exposant únicament funcions específiques d'interès.

La definició d'API és molt genèrica i podem trobar diverses API en molts contextos. En aquesta unitat, ens centrarem en l'ús de les API web per a l'adquisició de dades de serveis de tercers. Les **API web** es defineixen habitualment com un conjunt de peticions HTTP juntament amb l'especificació de l'estructura de les dades de les respostes, normalment en format JSON o XML.

L'ús de les API web està molt estès actualment per interactuar amb grans proveïdors de serveis a internet. Alguns exemples d'API populars són les de [Google maps](https://developers.google.com/maps/), [YouTube](https://developers.google.com/youtube/), [Spotify](https://developer.spotify.com/web-api/), [Twitter](https://dev.twitter.com/docs) o [Facebook](https://developers.facebook.com/docs/graph-api).

Diem que una API és RESTful (o, de vegades, simplement REST) quan compleix un conjunt de característiques, entre les quals destaca que no manté l'estat entre peticions. És a dir, tota la informació necessària per respondre una petició es troba en la petició individual (i no depèn de cap estat emmagatzemat pel servidor).

## 3.2 Peticions i respostes HTTP

Per interactuar amb una web API farem una petició HTTP. Al seu torn, el servidor ens respondrà amb un missatge de resposta HTTP. Les peticions i respostes HTTP s'estructuren en tres parts:

* Una línia inicial de petició, que inclou l'acció que cal realitzar (el mètode de la petició) i l'URL del recurs, a les peticions; i el codi d'estat i el missatge associat, en el cas de les respostes.
* La capçalera, que inclou metadades amb diverses finalitats, per exemple, per descriure'n el contingut, fer-ne l'autenticació o controlar les galetes (cookies).
* Una línia en blanc que separa la capçalera del cos.
* El cos, que pot estar buit o contenir-hi dades.

En la imatge següent es mostra un exemple simplificat d'una petició i una resposta HTTP:

In [ ]:
# Carreguem la biblioteca per mostrar les imatges
from IPython import display
display.Image("img/HTTP_request_response.webp")

La línia inicial de les respostes HTTP conté el codi d'estat, un nombre enter de tres xifres que informa sobre l'intent d'entendre i processar la petició HTTP. El primer dígit del nombre defineix el tipus de resposta. Actualment, existeixen cinc tipus reconeguts:

* 1xx: informacional.
* 2xx: èxit.
* 3xx: redirecció.
* 4xx: error del client.
* 5xx: error del servidor.

Així, quan naveguem per internet, normalment les nostres peticions es responen correctament, retornant el codi d'estat 200. De tant en tant ens trobem també amb errors del client. Per exemple, l'error 404 ens indica que la nostra màquina ha estat capaç de comunicar-se amb el servidor, però que el recurs que hem sol·licitat no existeix. Podem forçar aquest error accedint a URL no vàlides.

Les accions o mètodes més usats en interacció amb API web són:

* GET: permet obtenir informació del recurs especificat. Per exemple, quan s'introdueix un URL al navegador, el navegador envia una sol·licitud GET al servidor per obtenir la pàgina web corresponent.
* POST: permet enviar dades al recurs especificat. Per exemple, quan s'envia un formulari a una pàgina web, les dades del formulari s'envien al servidor utilitzant una sol·licitud POST.
* PUT: carrega dades actualitzant les ja existents al recurs especificat. Per exemple, si està treballant amb una API RESTful, es pot utilitzar una sol·licitud PUT per actualitzar un registre en una base de dades.
* DELETE: elimina informació del recurs especificat. Per exemple, en treballar amb una API RESTful, es pot utilitzar una sol·licitud DELETE per eliminar un registre d'una base de dades.


## 3.3. JSON i XML

Dos dels formats més habituals per incloure dades a les respostes de les API web són JSON i XML. Tots dos formats tenen diverses propietats en comú. En primer lloc, van ser dissenyats per ser llegits tant per humans com per ordinadors, la qual cosa els fa ideals en aquest context. En segon lloc, tots dos incorporen informació sobre l'estructura de les dades que codifiquen. Finalment, tots dos emmagatzemen les dades en text clar. No obstant això, tots dos presenten múltiples diferències.

El format **XML** (de l'anglès, Extensible Markup Language) és un llenguatge de marques que utilitza un conjunt d'etiquetes no predefinit. Els documents XML tenen un únic element arrel del qual poden penjar altres elements. Els elements es delimiten amb una etiqueta inicial i una etiqueta final. Vegem un exemple d'un document XML senzill:

```
<persona>
  <nombre>Yann</nombre>
  <cognoms>
    <cognom1>LeCun</cognom1>
    <cognom2>-</cognom2>
  </cognoms>
  <edat>56</edat>
</persona>
```

El format **JSON** (de l'anglès, Javascript Object Notation) és un subconjunt de la notació d'objectes Javascript. JSON es basa en dues estructures de dades, l'array i l'objecte, que serien equivalents a les llistes i diccionaris de Python que ja hem introduït.

Així, un array JSON és una llista ordenada de zero o de més valors, per exemple:

```
["data", "science", "course"]
```

En aquest cas, l'array està format per cadenes de caràcters.

Un objecte JSON és una col·lecció no ordenada de parells de clau i valor. Per exemple:

```
{
  "course": "Data Science",
  "year": 2017
}
```

Vegem un exemple de les dades que hem representat anteriorment en XML, fent servir ara el format JSON:

```
{
  "nombre": "Yann",
  "cognoms": {
    "cognom1": "LeCun",
    "cognom2": "-"
  },
  "edat" : 56
}
```

En aquest cas, hem fet servir un objecte amb tres claus: la primera té com a valor una cadena de caràcters, la segona té com a valor un altre objecte i la tercera té com a valor un enter.

La biblioteca json de Python ens ofereix algunes funcions molt útils per a treballar en aquest format. Per exemple, podem obtenir la representació JSON d'objectes Python o crear objectes Python a partir de la seva representació en JSON.

In [ ]:
# Construïm un diccionari d'exemple i mostrem el tipus de dades i el
# contingut de la variable.
diccionari_exemple = {"nom": "Yann",
                      "cognoms": {"cognom1": "LeCun", "cognom2": "-"},
                      "edat": 56}
print(type(diccionari_exemple))
print(diccionari_exemple)

# Construïm una llista d'exemple i mostrem el tipus de dades i el contingut de
# la variable.
llista_exemple = [1, 2, 3]
print(type(llista_exemple))
print(llista_exemple)

In [ ]:
# Importem la biblioteca json.
import json

# Mostrem la representació JSON del diccionari mitjançant la funció dumps()
json_dict = json.dumps(diccionari_exemple)
print(type(json_dict))
print(json_dict)

# Mostrem la representació JSON de la llista mitjançant la funció dumps()
json_list = json.dumps(llista_exemple)
print(type(json_list))
print(json_list)

Fixeu-vos que, en ambdós casos, obtenim una cadena de caràcters que ens representa, en format JSON, els objectes Python. Aquest procés es coneix com a **serialitzar** l'objecte.

També podem fer el procés invers (conegut com a **desserialitzar**), creant objectes Python (per exemple, llistes o diccionaris) a partir de cadenes de text en format JSON.

In [ ]:
# Desserialitzem la cadena 'json_dict' mitjançant la funció loads()
diccionario_ejemplo2 = json.loads(json_dict)
print(type(diccionario_ejemplo2))
print(diccionario_ejemplo2)

# Desserialitzem la cadena 'json_list' mitjançant la funció loads()
lista_ejemplo2 = json.loads(json_list)
print(type(lista_ejemplo2))
print(lista_ejemplo2)

Per millorar la llegibilitat de les dades que obtindrem de les APIs, definirem una funció que mostrarà cadenes JSON per pantalla formatades per millorar-ne la lectura. La funció acceptarà tant cadenes de caràcters amb contingut JSON com objectes Python, i mostrarà el contingut per pantalla.

A més, la funció rebrà un paràmetre opcional que ens permetrà indicar el nombre màxim de línies que cal mostrar. Així, podrem fer servir la funció per a visualitzar les primeres línies d'un JSON llarg, sense haver de mostrar el JSON complet per pantalla.

In [ ]:
# Defineix la funció 'json_print', que té un paràmetre obligatori 'json_data' i
# un paràmetre opcional 'limit' i no torna cap valor.
# La funció mostra per pantalla el contingut de la variable 'json_data' en
# format JSON, limitant el nombre de línies per mostrar si s'inclou el
# paràmetre 'limit'.
def json_print(json_data, limit=None):
    if isinstance(json_data, (str)):
        json_data = json.loads(json_data)
    nice = json.dumps(json_data, sort_keys=True,
                      indent=3, separators=(',', ':'))
    print("\n".join(nice.split("\n")[0:limit]))
    if limit is not None:
        print("[...]")

Vegem un exemple del resultat d'utilitzar la funció que acabem de
definir.

In [ ]:
# Mostra el valor de la variable 'json_exemple' amb la funció 'print'.
json_exemple = '{"nom": "Yann", "cognoms": {"cognom1": "LeCun", ' \
    '"cognom2": "-"}, "edat": 56}'
print(json_exemple)

In [ ]:
# Mostra el valor de la variable 'json_exemple' amb la funció 'json_print' que
# acabem de definir.
json_print(json_exemple)

In [ ]:
# Mostrem únicament les tres primeres línies.
json_print(json_exemple, 3)

## 3.4 Ús d'API de tercers en Python

### 3.4.1. Accés a API manualment

Podem utilitzar la biblioteca de Python [Requests](http://docs.python-requests.org/) per a realitzar peticions als webs API de manera manual. Per fer-ho, haurem d'accedir a la documentació de l'API amb la qual vulguem actuar, construir manualment les peticions per obtenir la informació desitjada i processar també manualment la resposta rebuda.

Vegem un exemple de petició HTTP a una API pública. El lloc http://postcodes.io/ ofereix una API de geolocalització sobre codis postals al Regne Unit. Llegint la documentació, podem veure que té un mètode GET amb l'URL
http://api.postcodes.io/postcodes/:código-postal, que ens retorna informació del codi postal especificat.

In [ ]:
# Importem la biblioteca.
import requests

# Fem una petició get a l'API, preguntant sobre el codi postal "E98 1TT".
# Fixeu-vos que el caràcter espai es codifica com a% 20 a la URL.
response = requests.get('http://api.postcodes.io/postcodes/E98%201TT')

# Mostrem la resposta rebuda.
print("Codi d'estat de la resposta: ", response.status_code, "\n")
print("Capçalera de la resposta: ")
json_print(dict(response.headers))
print("\nCos de la resposta: ")
json_print(response.text)

Com podem veure, l'estat de la resposta és 200, la qual cosa [ens indica](https://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html) que la petició s'ha processat correctament. Entre altres camps, la capçalera de la resposta inclou el tipus de contingut que trobarem al cos, que serà un text en format JSON. Finalment, el cos de la resposta inclou dades sobre el codi postal consultat. Per exemple, podem veure que correspon a la nació d'Anglaterra (concretament, a la ciutat de Londres).

Fixeu-vos que podem visualitzar també la resposta accedint a la [mateixa URL](http://api.postcodes.io/postcodes/E98%201TT) amb un navegador web. En aquest cas, es poden instal·lar extensions específiques que gestionin la visualització millorada del JSON retornat (per exemple, [JSONView](https://chrome.google.com/webstore/detail/jsonview/chklaanhfefbnpoihckbnefhakgolnmc) per Chrome o Firefox).

### 3.4.2 Accés a API amb biblioteques de Python

Encara que podríem fer servir aquest mètode per interactuar amb qualsevol API HTTP, la veritat és que quan la complexitat de les funcions disponibles incrementa (per exemple, en incloure autenticació) pot no resultar gaire pràctic. Quan vulguem accedir a les API populars, normalment trobarem que ja existeixen biblioteques de Python dissenyades per interactuar amb aquestes API, de manera que podrem obtenir dades sense necessitat de gestionar les peticions HTTP manualment.

Per exemple, Spotify, la famosa plataforma de músic, té una API que permet executar tasques com buscar i obtenir informació sobre artistes, àlbums, pistes, llistes de reproducció, obtenir recomanacions de música, accedir a dades d'àudio, entre altres coses. Per interaccionar amb aquesta API, utilitzarem la biblioteca de Python [Spotipy](https://spotipy.readthedocs.io/en/2.22.1/).

#### Autenticació amb l'API de Spotify

Per utilitzar l'API de Spotify, necessiteu obtenir credencials d'autenticació. Per això, us haureu de registrar com a desenvolupador al lloc web de Spotify i crear una aplicació. Després de crear-la, obtindreu un *Client ID* i un *Client Secret* que s'utilitzaran per autenticar-se a l'API.

En aquesta secció, descriurem com obtenir credencials per accedir a l'API de Spotify.

Per començar, cal disposar d'un compte a Spotify (gratuït o prèmium). Podeu utilitzar el vostre compte personal, si ja en disposeu d'un, per a sol·licitar els permisos de desenvolupador que ens permetran interactuar amb l'API. En cas contrari (o si preferiu no fer servir el vostre compte personal), podeu crear-vos un compte nou.

El procés per a crear un nou compte és molt senzill:
1. Accedir a [Spotify] (https://open.spotify.com/).
2. Prémer sobre *Sign up* i seguir les indicacions per completar el registre.

Després, caldrà crear una nova App i obtenir les credencials per connectar-se a la API. Per fer-ho, cal seguir els següents passos explicats al [tutorial](https://developer.spotify.com/documentation/web-api/tutorials/getting-started):

1. Accediu al [panell de desenvolupadors de Spotify](https://developer.spotify.com/documentation/web-api).
2. Accediu al dashboard de desenvolupadors (https://developer.spotify.com/dashboard). En aquest pas és possible que us faci verificar el correu electrònic.

**Creació App**
1. Cliqueu sobre *Create App* per a crear una nova App.
2. Ompliu el formulari amb els detalls de l'aplicació. En concret, necessitareu proporcionar com a mínim els camps:
    * *App name*
    * *Application description*
    * *Redirect URl*
    * *APIs used*
Poseu el nom i la descripció que vulgueu per l'App. En l'opció de *Redirect URl* poseu: http://localhost:3000. Per acabar, marqueu l'opció *Web API*.

3. Accepteu els termes i les condicions i cliqueu *Save*.

**Tokens d'accés**

A continuació, s'ha de demanar un *token* d'accés.
1. En el panell de l'App que acabeu de crear, aneu a l'apartat de *Settings*. Allà trobareu el *Client ID* i el *Client Secret* (aquest últim apareix en clicar sobre *View client secret*).

Quan tingueu les credencials, ja podeu començar a utilitzar l'API de Spotify a Python.

#### La biblioteca Spotipy

[Spotipy](https://pypi.org/project/spotipy/) ens permet interactuar amb l'API de Spotify d'una manera senzilla, ja que encapsula els mètodes HTTP de l'API en mètodes de Python, que poden ser cridats directament. Trobareu la documentació de la biblioteca en el següent [enllaç](https://spotipy.readthedocs.io/en/2.22.1/).

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    # Primer de tot, hem d'instal·lar la biblioteca spotipy
    %pip install spotipy

In [ ]:
# Importem la biblioteca spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

try:
    from google.colab import userdata
    spotipy_client_id = userdata.get('SPOTIPY_CLIENT_ID')
    spotipy_client_secret = userdata.get('SPOTIPY_CLIENT_SECRET')
except:
    from dotenv import load_dotenv
    import os
    load_dotenv('.env')
    spotipy_client_id = os.getenv("SPOTIPY_CLIENT_ID")
    spotipy_client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")

# Ens connectem a l'API de Spotify amb les nostres credencials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id=spotipy_client_id,
    client_secret=spotipy_client_secret))

# Buscarem informació sobre un artista
artist_name = 'Metallica'
results = sp.search(q='artist:' + artist_name, type='artist')
artists = results['artists']['items']

if artists:
    artist = artists[0]
    print("Nom de l'artista:", artist['name'])
else:
    print("No s'ha trobat l'artista.")

Fixeu-vos que, en aquest cas, no hem hagut de gestionar les peticions HTTP manualment: la biblioteca ho ha fet per nosaltres de forma transparent.

A més, les funcions de la biblioteca ens tornen directament objectes Python, que poden ser usats com qualsevol altre. Per exemple, podem seleccionar només una part de les respostes de les API segons el nostre interès (en l'exemple anterior, hem seleccionat l'identificador i el nom d'usuari directament usant l'objecte `artist`). Vegem alguns exemples més d'atributs que hem recuperat de l'artista:

In [ ]:
# Mostrem alguns atributs de l'artista
print("El número de seguidors és: {}".format(artist['followers']['total']))
print("El gènere és: {}".format(artist['genres'][0]))

# 4 Obtenció de dades a partir de *web crawling*

De vegades ens interessarà capturar dades que es troben a internet, però per les quals no existeix una API que ens permeti accedir-hi de forma estructurada. En aquests casos, una alternativa és programar una aranya (en anglès, un _**web crawler**_), un programa que analitza pàgines web de forma automàtica a la recerca del contingut d'interès.

El procediment essencial d'un _web crawler_ consisteix a explorar una determinada pàgina web a la recerca de dades d'interès, que s'emmagatzemaran per al seu ús posterior, i enllaços a altres pàgines web d'interès, que seran explorades posteriorment pel mateix _crawler_, a la recerca de noves dades d'interès i noves pàgines.

Per obtenir tant les dades com els enllaços d'interès, el _web crawler_ utilitza un analitzador sintàctic (en anglès, _**parser**_), que processa l'HTML de la pàgina web i n'extreu les dades.

## 4.1 HTML

El format **HTML** (de l'anglès, _Hypertext Markup Language_) és el llenguatge de marques estàndard per descriure la presentació de pàgines web. De la mateixa manera que XML, utilitza (majoritàriament) una etiqueta inicial i una final per indicar elements. A diferència d'XML, les etiquetes es troben prefixades per un estàndard.

A més de senyalitzar l'inici i el final d'un element, les etiquetes HTML poden incloure atributs, que permeten proporcionar informació addicional sobre els elements.

Vegem un exemple d'un document HTML senzill:

```
<html>
  <head>
    <title>El título de la página</title>
  </head>
  <body>
    <div class="clase1" id =”id1”>
       <p> Un texto </p>
    </div>
    <div class="clase1" id =”id2”>
       <p> Otro texto </p>
    </div>
  </body>
</html>
```

## 4.2 _Web crawling_ en Python amb Scrapy

[Scrapy](https://scrapy.org/) és una biblioteca de Python que proveeix d'un _framework_ per a l'extracció de dades de pàgines web. Scrapy és molt complet i disposa de múltiples funcionalitats, però en veurem un exemple senzill d'ús.

Suposem que volem obtenir un llistat dels àmbits de les titulacions que ofereix la UOC. La UOC no ofereix una API amb aquesta informació, però sí que podem trobar-la a la pàgina <https://www.uoc.edu/portal/es/universitat/estudis-centres-recerca/index.html>.

De totes maneres, no volem anar copiant manualment els noms de tots els àmbits per obtenir el llistat d'interès, per la qual cosa desenvoluparem un petit _crawler_ que obtingui aquestes dades per nosaltres.

Ja tenim identificada l'URL que volem explorar (<https://www.uoc.edu/portal/es/universitat/estudis-centres-recerca/index.html>), així que només caldrà identificar on es troben les dades d'interès dins de la pàgina. Per fer-ho, en primer lloc, ens fixarem en algun títol de grau que aparegui a la pàgina, per exemple, "Estudios de Artes y Humanidades" o "Estudios de Ciencias de la Información y de la Comunicación". Seguidament accedirem al codi font de la pàgina (podem fer servir la combinació de tecles `CTRL + u` als navegadors Firefox o Chrome) i buscarem els noms dels graus que hem vist anteriorment:

```
<h3 id="card-title-1682057768418" class="h3 card-title">
  <a href="/portal/es/estudis_arees/arts_humanitats/index.html" target="_blank">
        Estudios de Artes y Humanidades
    <span class="absolute-link" aria-hidden="true">
    </span>
  </a>
</h3>
```

Com es pot apreciar, les dades que volem recopilar (els noms dels àmbits que ofereix la UOC) es troben en forma de text dintre de l'element h3 que té l'atribut classe fixat a «h3 card-title».

Per a indicar que volem seleccionar aquestes dades, utilitzarem la sintaxi XPath. En concret, utilitzarem l'expressió

que ens indica que volem seleccionar totes les etiquetes `<h3>` que tinguin com a atribut classe el valor «h3 card-title» i extreure'n el text. Com hi ha diversos elements amb aquestes característiques, especifiquem que els volem tots amb `//*/`. Amb això ja podem programar la nostra aranya perquè n'extregui les dades d'interès.

L'estructura d'un _crawler_ amb Scrapy ve prefixada. En el nostre cas, només serà necessari definir una aranya i incloure un _parser_ que extregui les dades dels àmbits i que disposi de l'URL d'inici.

Abans de veure l'estructura d'un crawler, hem d'instal·lar Scrapy a colab amb:

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    %pip install scrapy

In [ ]:
# Importem Scrapy.
import scrapy
from scrapy.crawler import CrawlerProcess


class uoc_spider(scrapy.Spider):

    # Assignem un nom a l'aranya.
    name = "uoc_spider"

    # Indiquem l'URL que volem analitzar en primer lloc.
    start_urls = [
        "https://www.uoc.edu/portal/es/universitat/estudis-centres-recerca/index.html"
    ]

    # Definim l'analitzador.
    def parse(self, response):
        # Extraiem el títol del grau.
        for ambito in response.xpath('//h3[@class="h3 card-title"]//*/text()'):
            yield {
                'Ambito': ambito.extract()
            }

Un cop definida l'aranya, llançarem el _crawler_ indicant que volem que usi l'aranya `uoc_spider` que acabem de definir:

In [ ]:
if __name__ == "__main__":

    # Creem un crawler:
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'DOWNLOAD_HANDLERS': {'s3': None},
        'LOG_ENABLED': True,
        'LOG_LEVEL': 'INFO'
    })

    # Inicialitzem el crawler amb la nostra aranya:
    process.crawl(uoc_spider)

    # Llancem l'aranya:
    process.start()

L'execució de Scrapy mostra un registre detallat amb tots els esdeveniments que han anat passant, fet que és molt útil per identificar problemes, sobretot en captures complexes. En el nostre cas, a més, podem veure com s'han extret els noms dels diferents àmbits de les titulacions de la UOC.

## 4.3 _Web crawling_ en Python amb BeautifulSoup

Una altra opció per poder extreure dades de pàgines web a partir dels seus fitxers HTML o XML és utilitzar la biblioteca Beautiful Soup. És una eina popular per fer web scraping, ja que té una sintaxi més senzilla que Scrapy.

La biblioteca Beautiful Soup analitza l'HTML (o XML) d'una pàgina web i el converteix en un conjunt d'objectes Python que podeu navegar i manipular fàcilment. Permet cercar, filtrar i modificar els elements HTML d'una manera senzilla i eficient.

A continuació repetirem l'extracció que hem fet a l'apartat anterior amb `Scrapy`, però amb `BeautifulSoup`.

Primer de tot, utilitzem la biblioteca `requests` per obtenir el contingut HTML de la pàgina web que volem analitzar. A continuació, creem un objecte `BeautifulSoup` passant el contingut HTML que hem obtingut i un parser (en aquest cas, en tractar-se d'un fitxer HTML especifiquem html.parser).

Si mostrem per pantalla el contingut de l'objecte `soup` veurem tota la informació de la pàgina HTML.

In [ ]:
# Carreguem les biblioteques necessàries
import requests
from bs4 import BeautifulSoup
import pprint

# Especifiquem la URL
URL = "https://www.uoc.edu/portal/es/universitat/estudis-centres-recerca/" \
    "index.html"

# obtenim la resposta y el contingut de la URL
response = requests.get(URL)
# print("----------------- header -----------------")
# pprint.pp(dict(response.headers))
soup = BeautifulSoup(response.content, 'html.parser')

# Descomentar per veure el contingut de soup
# print("----------------- content -----------------")
# print(soup.prettify())

Com ja hem vist a l'anterior apartat, la informació que ens interessa està dintre de l'element h3 card-title. També sabem que hi ha diferents àmbits que volem recollir, i que tots segueixen la mateixa estructura (el nom es troba en forma de text dintre d'aquest element h3). Per això, utilitzarem la funció `find_all` per guardar aquests elements en una llista `ambits`.

Si mostrem el contingut d'aquesta llista, podem veure que cada element conté la informació HTML d'un àmbit.

In [ ]:
ambits = soup.find_all('h3', class_='h3 card-title')
print(ambits[0])
print(ambits[1])

Però en aquest cas només volem el nom, així que podem iterar sobre la llista i mostrar el text de cada element.

In [ ]:
for g in ambits:
    # .strip() es fa servir per eliminar espais innecessaris
    ambit = g.text.strip()
    print(ambit)

# 5 Exercicis i preguntes teòriques

La part avaluable d'aquesta unitat consisteix en el lliurament d'un fitxer IPython Notebook amb extensió IPYNB que contindrà els diferents exercicis i les preguntes teòriques que s'han de contestar. Trobareu el fitxer (`prog_datasci_5_python_entrega.ipynb`) amb les activitats a la mateixa carpeta que aquest notebook que esteu llegint.

## 5.1 Instruccions importants

És molt important que a l'hora de lliurar el fitxer Notebook amb les vostres activitats us assegureu que:

    1. Les vostres solucions siguin originals. Esperem no detectar-hi còpia directa entre estudiants.
    2. Tot el codi estigui correctament documentat. El codi sense documentar equivaldrà a un 0.
    3. El fitxer comprimit que lliureu és correcte (conté les activitats de la PAC que heu de lliurar).
    
Per fer el lliurament, heu d'anar a la carpeta del drive Colab Notebooks, clicant botó dret a la PAC en qüestió i fent Download. D'aquesta manera baixareu la carpeta de la PAC comprimida en zip. Aquest és el fitxer que heu de pujar al campus de virtual de l'assignatura.


# 6 Bibliografia

Per obtenir més informació sobre els formats de dades presentats en aquesta unitat, podeu consultar el w3schools ([JSON](https://www.w3schools.com/js/js_json_intro.asp), [XML](https://www.w3schools.com/xml/) i [HTML](https://www.w3schools.com/html/)).

# 7 Annex: L'API de googlemaps

Aquest annex conté un exemple addicional d'accés a API amb biblioteques de Python. En concret, l'exemple mostra com accedir a l'API de googlemaps. En el passat, l'ús d'aquesta API era gratuït, però actualment l'ús de l'API té múltiples restriccions i, tot i que es poden fer algunes peticions gratuïtament, és necessari proporcionar dades de la nostra targeta de crèdit per poder interactuar amb l'API. Podeu revisar el codi d'aquest exemple per tenir un exemple més de l'ús de biblioteques per accedir a les API, o bé crear un compte a la plataforma de google developers i provar els exemples proporcionats. En aquest últim cas, recordeu revisar la política de cobrament de googlemaps, per assegurar que no sobrepasseu el límit gratuït, abans de dur a terme les proves.

Google maps disposa d'un [conjunt d'API](https://developers.google.com/maps/) molt populars que permeten, entre d'altres, obtenir les coordenades geogràfiques d'una adreça, aconseguir indicacions per desplaçar-se d'un punt a un altre, o adquirir dades sobre l'elevació del terreny a qualsevol punt del món. La biblioteca [googlemaps](https://googlemaps.github.io/google-maps-services-python/docs/2.4.6/) integra peticions a l'API de Google en codi Python.

Per fer servir les API de Google Maps, cal registrar un usuari i obtenir una clau d'autenticació, que adjuntarem a les peticions que es facin contra l'API. A més, haurem d'especificar quines APIs concretes farem servir.

A l'exemple següent, farem aquests tres passos per obtenir la clau d'autenticació:

1.  Crearem un projecte a la plataforma de Google Developers.
2.  Activarem les API desitjades.
3.  Sol·licitarem credencials d'accés.

En primer lloc, crearem un nou projecte a l'entorn de desenvolupadors de google. Ens dirigirem a: <https://console.developers.google.com/apis/library> i farem clic sobre «Project: New project». Assignarem un nom qualsevol al projecte i confirmarem la creació clicant sobre «Create».

Un cop creat el projecte, activarem les API que farem servir. Primer, seleccionarem l'API de geocodificació ([*Google Maps Geocoding API*](https://console.developers.google.com/apis/api/geocoding_backend)), que es troba a la categoria *Google Maps APIs* (és possible que hagueu de prémer sobre el botó «more» per veure la llista completa de les API). Farem clic sobre «Enable» per activar-la.

Repetirem el procés per a l'API d'adreces ([*Google Maps Directions API*](https://console.developers.google.com/apis/api/directions_backend)), que es troba també a la categoria *Google Maps APIs*.

Finalment, farem clic sobre el menú «Credentials», indicarem «Create credentials» i escollirem «API Key». Ens apareixerà una finestra amb una cadena de caràcters que representa la nostra clau. Perquè l'exemple següent funcioni, **cal que assigneu a la variable `api_key` el valor de la vostra clau**.

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    %pip install googlemaps

In [ ]:
from datetime import datetime
# Importem la biblioteca googlemaps, que interactuarà amb l'API de Google Maps.
import googlemaps
# Importem la biblioteca logging per a controlar els missatges de registre.
import logging

# Establim el nivell de logging per evitar mostrar missatges informatius.
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.WARNING)
logging.getLogger("urllib3.util.retry").setLevel(logging.WARNING)
logging.getLogger("googlemaps.client").setLevel(logging.WARNING)

try:
    from google.colab import userdata
    google_maps_api_key = userdata.get("GOOGLE_MAPS_API_KEY")
except:
    from dotenv import load_dotenv
    import os
    load_dotenv('.env')
    google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")

# Inicialitzem el client, indicant la clau d'autenticació,
gmaps = googlemaps.Client(key=google_maps_api_key)

En primer lloc, farem servir l'[API de geocodificació](https://developers.google.com/maps/documentation/geocoding/start) per obtenir dades d'una adreça per mitjà del mètode [Geocode](https://googlemaps.github.io/google-maps-services-python/docs/2.4.6/#googlemaps.Client.geocode)
del client de Google Maps que ens ofereix la biblioteca (emmagatzemat a la variable `gmaps`).

In [ ]:
# Utilitzem l'API de geocodificació per obtenir dades d'una adreça.
geocode_result = gmaps.geocode('Rambla del Poblenou, 156, Barcelona')
print("------ Resultat de geocode ------")
json_print(geocode_result, 20)

Un altre exemple de l'ús de l'[API de geocodificació](https://developers.google.com/maps/documentation/geocoding/start) utilitza el mètode [reverse_geocode](https://googlemaps.github.io/google-maps-services-python/docs/2.4.6/#googlemaps.Client.reverse_geocode) per obtenir informació sobre unes coordenades geogràfiques concretes:

In [ ]:
# Obtenim dades sobre unes coordenades geogràfiques.
reverse_geocode_result = gmaps.reverse_geocode((41.2768089, 1.9884642))
print("------ Resultat de reverse geocode ------")
json_print(reverse_geocode_result, 20)

L'exemple següent interactua amb l'[API d'adreces](https://developers.google.com/maps/documentation/directions/)
fent servir el mètode [_directions_](https://googlemaps.github.io/google-maps-services-python/docs/2.4.6/#googlemaps.Client.directions) de la biblioteca googlemaps de Python, per obtenir indicacions de
desplaçament entre dos punts.

In [ ]:
# Obtenim indicacions sobre com anar d'una adreça a una altra, considerant el
# trànsit del moment actual.
now = datetime.now()
directions_result = gmaps.directions("Carrer Colom, 114, Terrassa",
                                     "Carrer Sant Antoni, 1, Salt",
                                     mode="transit",
                                     departure_time=now)
print("------ Resultat de directions ------")
json_print(directions_result, 15)

Fixeu-vos que, en aquest cas, no hem hagut de gestionar les peticions HTTP manualment: la biblioteca ho ha fet per nosaltres de manera transparent.

A més, les funcions de la biblioteca ens tornen directament objectes Python, que es poden fer servir com qualsevol altre. Per exemple, podem seleccionar només una part de les respostes de les API segons el nostre interès:

In [ ]:
# Mostrem les claus del diccionari que retorna la crida a geocode.
geocode_result[0].keys()

In [ ]:
# Mostrem únicament les coordenades geogràfiques de la direcció d'interès.
geocode_result[0]["geometry"]["location"]

In [ ]:
# Mostrem les localitzacions properes a les coordenades geogràfiques que hem preguntat amb 'reverse_geocode',
# tot imprimint-ne les coordenades exactes i l'adreça.
for result in reverse_geocode_result:
    print(result["geometry"]["location"], result["formatted_address"])

In [ ]:
# Mostrem únicament la distància del trajecte entre els dos punts preguntats a l'API d'adreces.
print(directions_result[0]["legs"][0]["distance"])

# Autors

- Autor original **Cristina Pérez Solà**, 2017.
- Actualitzat per **Cristina Pérez Solà**, 2019.
- Actualitzat per **Alexandra Abós Ortega**, 2024.